---

This part of the file is not present in the JupyterLab file since this part is the local execution

In [1]:
%pip install -r requirements.txt

  Using cached py4j-0.10.9.7-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached pyspark-3.5.5-py2.py3-none-any.whl
Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


---

In [2]:
from pyspark import SparkContext

# Initialize SparkContext
sc = SparkContext("local", "AirbnbPrices")
sc

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/26 22:31:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local appName=AirbnbPrices>

In [4]:
# Load data from file
rdd = sc.textFile('preus.txt')
# Parse data to float and remove headers and non numeric values
prices_rdd = rdd.filter(lambda x: x.strip().isdigit()).map(lambda x: float(x.strip()))

In [5]:
# Number of accommodations
num_accommodations = prices_rdd.count()
print(f'Number of accomodations: {num_accommodations}')

Number of accomodations: 1048253


In [6]:
# Min and Max price value
min_price = prices_rdd.min()
max_price = prices_rdd.max()
print(f'Minimun: {min_price} - Maximum: {max_price}')

Minimun: 10.0 - Maximum: 23229.0


In [7]:
# Mean and standard deviation
mean_price = prices_rdd.mean()
stdev_price = prices_rdd.stdev()
print(f'Mean: {mean_price} - Standard deviation: {stdev_price}')

Mean: 436.0350125399073 - Standard deviation: 1188.3904253821997


In [8]:
# Update price values
new_price_rdd = prices_rdd.map(lambda x: x * 1.02 if x < 1000 else x * 1.03)

# New mean and standard deviation
new_mean_price = new_price_rdd.mean()
new_stdev_price = new_price_rdd.stdev()
print(f'NEW Mean: {new_mean_price} - NEW Standard deviation: {new_stdev_price}')

NEW Mean: 446.8178788517739 - NEW Standard deviation: 1224.0973352938518


In [9]:
# Count accommodations by groups
#! I'll suppose that this part makes reference to the updated values 
price_groups = {
	'Group 1': new_price_rdd.filter(lambda x: x <= 150).count(),
	'Group 2': new_price_rdd.filter(lambda x: 151 <= x <= 300).count(),
	'Group 3': new_price_rdd.filter(lambda x: 301 <= x <= 500).count(),
	'Group 4': new_price_rdd.filter(lambda x: x > 500).count()
}
for group, count in price_groups.items():
	print(f'{group}: {count}')

Group 1: 396986
Group 2: 300880
Group 3: 145719
Group 4: 199829


In [10]:
# Stop SparkContext
sc.stop()